```{contents}
```
## Composite Node

A **Composite Node** in LangGraph is a **node whose internal implementation is itself a graph (subgraph)**.
It enables **hierarchical workflow design**, where complex logic is encapsulated as a reusable, higher-level operation.

In other words:

> **A composite node = a graph used as a node inside another graph**

---

### **1. Why Composite Nodes Exist**

Real systems quickly exceed the complexity of flat graphs.
Composite nodes solve this by introducing **modularity and abstraction**.

| Problem                        | Composite Node Solution        |
| ------------------------------ | ------------------------------ |
| Large graphs become unreadable | Break into logical components  |
| Repeated workflows             | Reusable subgraphs             |
| Difficult testing              | Isolate and test subgraphs     |
| Complex control logic          | Encapsulate into sub-pipelines |
| Multi-team development         | Contract-based interfaces      |

---

### **2. Conceptual Model**

```
Main Graph
   |
   ├── Preprocess
   ├── Research   ← Composite Node
   │      ├── Search
   │      ├── Filter
   │      ├── Summarize
   │      └── Verify
   └── Final Answer
```

To the outer graph, **Research** appears as a single node.
Internally, it is a full LangGraph workflow.

---

### **3. Formal Definition**

A composite node is implemented by compiling a subgraph and registering it as a node:

```python
research_graph = research_builder.compile()
builder.add_node("research", research_graph)
```

LangGraph automatically:

* Forwards state into the subgraph
* Executes the internal graph
* Merges the resulting state back

---

### **4. State Interface Design**

Composite nodes operate on **shared state contracts**.

```python
class MainState(TypedDict):
    question: str
    research_notes: str
    answer: str
```

The subgraph must respect the same schema (or a compatible subset).

---

### **5. Working Example**

#### Step 1 — Build Subgraph

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class ResearchState(TypedDict):
    question: str
    research_notes: str

def search(state):
    return {"research_notes": f"Info about {state['question']}"}

builder = StateGraph(ResearchState)
builder.add_node("search", search)
builder.set_entry_point("search")
builder.add_edge("search", END)

research_graph = builder.compile()
```

#### Step 2 — Use Subgraph as Composite Node

```python
class MainState(TypedDict):
    question: str
    research_notes: str
    answer: str

def answer(state):
    return {"answer": f"Final answer using {state['research_notes']}"}

main = StateGraph(MainState)
main.add_node("research", research_graph)
main.add_node("answer", answer)

main.set_entry_point("research")
main.add_edge("research", "answer")
main.add_edge("answer", END)

app = main.compile()
print(app.invoke({"question": "LangGraph"}))
```

---

### **6. Variants of Composite Nodes**

| Variant                 | Use Case                 |
| ----------------------- | ------------------------ |
| Sequential composite    | Linear internal workflow |
| Conditional composite   | Internal branching       |
| Cyclic composite        | Iterative reasoning      |
| Parallel composite      | Concurrent subflows      |
| Agent composite         | Multi-agent systems      |
| Human-in-loop composite | Compliance workflows     |

---

### **7. Advanced Patterns Enabled**

| Pattern                   | Description                   |
| ------------------------- | ----------------------------- |
| Hierarchical agents       | Supervisor → Worker subgraphs |
| Domain pipelines          | Separate subgraphs per domain |
| Self-improving systems    | Composite + reflection loops  |
| Large-scale orchestration | Multi-layer graph systems     |

---

### **8. Production Considerations**

| Aspect            | Practice                          |
| ----------------- | --------------------------------- |
| Testing           | Unit test subgraphs independently |
| Versioning        | Version composite nodes           |
| Contracts         | Freeze state schema               |
| Observability     | Trace inside subgraphs            |
| Failure isolation | Localized retries & rollback      |

---

### **9. Mental Model**

Composite nodes allow you to design LangGraph systems like **software architecture**:

> Functions → Modules → Services → Systems

They are the foundation of **scalable, maintainable, production-grade LangGraph applications**.

### Demonstration

In [1]:
from typing import TypedDict

class MainState(TypedDict):
    question: str
    research_notes: str
    answer: str

from langgraph.graph import StateGraph, END

def search_node(state: MainState):
    return {"research_notes": f"Research data for: {state['question']}"}

def verify_node(state: MainState):
    notes = state["research_notes"] + " (verified)"
    return {"research_notes": notes}

research_builder = StateGraph(MainState)

research_builder.add_node("search", search_node)
research_builder.add_node("verify", verify_node)

research_builder.set_entry_point("search")
research_builder.add_edge("search", "verify")
research_builder.add_edge("verify", END)

research_graph = research_builder.compile()

In [2]:
def answer_node(state: MainState):
    return {"answer": f"Final answer based on {state['research_notes']}"}

main_builder = StateGraph(MainState)

main_builder.add_node("research", research_graph)   # Composite Node
main_builder.add_node("answer", answer_node)

main_builder.set_entry_point("research")
main_builder.add_edge("research", "answer")
main_builder.add_edge("answer", END)

app = main_builder.compile()


In [3]:
result = app.invoke({"question": "What is LangGraph?"})
print(result)


{'question': 'What is LangGraph?', 'research_notes': 'Research data for: What is LangGraph? (verified)', 'answer': 'Final answer based on Research data for: What is LangGraph? (verified)'}
